# Population et conditions sociales

## Population (niveau national) (t_demo_pop)

### Population par citoyenneté - Etrangers (tps00157)

#### Description de l'API
Nombre de personnes n'ayant pas la nationalité du pays dans lequel elles résident (le pays déclarant), y compris les citoyens d'autres États membres de l'UE, les citoyens de pays non membres de l'UE ainsi que les apatrides, résidant habituellement dans le pays déclarant au 1er janvier de l'année concernée.

### Dimensions du Dataset

- **Fréquence Temporelle**
  - Type : Annuel

- **Pays de Nationalité**
  - Type : Étranger et apatrides

- **Unité de Mesure**
  - Type : Nombre

- **Entité Géopolitique (reporting)**
  - **Entités** :
    - Union européenne - 27 pays (depuis 2020)
    - Zone euro – 20 pays (depuis 2023)
    - Zone euro - 19 pays (2015-2022)
    - Belgique
    - Bulgarie
    - République tchèque
    - Danemark
    - Allemagne
    - Estonie
    - Irlande
    - Grèce
    - Espagne
    - France
    - France métropolitaine
    - Croatie
    - Italie
    - Chypre
    - Lettonie
    - Lituanie
    - Luxembourg
    - Hongrie
    - Malte
    - Pays-Bas
    - Autriche
    - Pologne
    - Portugal
    - Roumanie
    - Slovénie
    - Slovaquie
    - Finlande
    - Suède
    - Islande
    - Liechtenstein
    - Norvège
    - Suisse
    - Royaume-Uni
    - Bosnie-Herzégovine
    - Monténégro
    - Moldavie
    - Macédoine du Nord
    - Géorgie
    - Albanie
    - Serbie
    - Türkiye
    - Ukraine
    - Kosovo*
    - Andorre
    - Biélorussie
    - Monaco
    - Russie
    - Saint-Marin
    - Arménie
    - Azerbaïdjan

- **Temps**
  - **Années** :
    - 2013
    - 2014
    - 2015
    - 2016
    - 2017
    - 2018
    - 2019
    - 2020
    - 2021
    - 2022
    - 2023
    - 2024


In [1]:
#Préparation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
import statsmodels.api as sm
from scipy import stats
import requests
import eurostat


In [2]:
df = eurostat.get_data_df('tps00157')
df.columns

Index(['freq', 'citizen', 'age', 'unit', 'sex', 'geo\TIME_PERIOD', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', '2023'],
      dtype='object')

In [3]:
df.head()

,freq,citizen,age,unit,sex,geo\TIME_PERIOD,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,A,FOR_STLS,TOTAL,NR,T,AT,946610.0,999095.0,1060446.0,1139736.0,1260188.0,1333239.0,1385823.0,1427105.0,1473453.0,1517372.0,1572316.0,1714698.0
1,A,FOR_STLS,TOTAL,NR,T,BE,1197598.0,1222741.0,1237696.0,1272751.0,1327421.0,1346358.0,1366070.0,1400238.0,1437580.0,1454039.0,1488896.0,1589004.0
2,A,FOR_STLS,TOTAL,NR,T,BG,39432.0,44506.0,53253.0,63879.0,71491.0,76562.0,82370.0,91243.0,100768.0,108438.0,63861.0,83715.0
3,A,FOR_STLS,TOTAL,NR,T,CH,1815063.0,1869070.0,1936412.0,1997179.0,2047200.0,2099319.0,2124540.0,2146381.0,2173429.0,2208864.0,2242343.0,2294345.0
4,A,FOR_STLS,TOTAL,NR,T,CY,172427.0,170076.0,159336.0,144599.0,139606.0,140384.0,149168.0,155610.0,161005.0,165561.0,170061.0,183505.0


In [12]:
# Dictionnaire des abréviations et noms des pays en français
eurostat_dictionary = {
    
  'BE': 'Belgique',
    'BG': 'Bulgarie',
    'CZ': 'République tchèque',
    'DK': 'Danemark',
    'DE': 'Allemagne',
    'EE': 'Estonie',
    'IE': 'Irlande',
    'EL': 'Grèce',
    'ES': 'Espagne',
    'FR': 'France',
    'HR': 'Croatie',
    'IT': 'Italie',
    'CY': 'Chypre',
    'LV': 'Lettonie',
    'LT': 'Lituanie',
    'LU': 'Luxembourg',
    'HU': 'Hongrie',
    'MT': 'Malte',
    'NL': 'Pays-Bas',
    'AT': 'Autriche',
    'PL': 'Pologne',
    'PT': 'Portugal',
    'RO': 'Roumanie',
    'SI': 'Slovénie',
    'SK': 'Slovaquie',
    'FI': 'Finlande',
    'SE': 'Suède',
    'IS': 'Islande',
    'LI': 'Liechtenstein',
    'NO': 'Norvège',
    'CH': 'Suisse',
    'UK': 'Royaume-Uni',
    'ME': 'Monténégro',
    'MD': 'Moldavie',
    'MK': 'Macédoine du Nord',
    'AL': 'Albanie',
    'RS': 'Serbie',
    'TR': 'Türkiye',
    'AD': 'Andorre',
    
}
# Renommer la colonne 'geo\\time' en 'geo'
df.rename({'geo\\TIME_PERIOD': 'geo'}, inplace=True, axis=1)
# Créer une nouvelle colonne 'pays' avec les noms des pays
df['pays'] = df['geo'].replace(eurostat_dictionary)
print(df.head())

  freq   citizen    age unit sex geo       2012       2013       2014  \
0    A  FOR_STLS  TOTAL   NR   T  AT   946610.0   999095.0  1060446.0   
1    A  FOR_STLS  TOTAL   NR   T  BE  1197598.0  1222741.0  1237696.0   
2    A  FOR_STLS  TOTAL   NR   T  BG    39432.0    44506.0    53253.0   
3    A  FOR_STLS  TOTAL   NR   T  CH  1815063.0  1869070.0  1936412.0   
4    A  FOR_STLS  TOTAL   NR   T  CY   172427.0   170076.0   159336.0   

        2015       2016       2017       2018       2019       2020  \
0  1139736.0  1260188.0  1333239.0  1385823.0  1427105.0  1473453.0   
1  1272751.0  1327421.0  1346358.0  1366070.0  1400238.0  1437580.0   
2    63879.0    71491.0    76562.0    82370.0    91243.0   100768.0   
3  1997179.0  2047200.0  2099319.0  2124540.0  2146381.0  2173429.0   
4   144599.0   139606.0   140384.0   149168.0   155610.0   161005.0   

        2021       2022       2023      pays  
0  1517372.0  1572316.0  1714698.0  Autriche  
1  1454039.0  1488896.0  1589004.0  Belg

In [13]:
# Pivoter le DataFrame jusqu'à 2023
df_melted = df.melt(id_vars=['pays', 'freq', 'citizen', 'age', 'unit', 'sex', 'geo'], 
                    value_vars=[str(year) for year in range(2012, 2024)],  # Changer 2025 à 2024
                    var_name='année', value_name='valeur')

# Pivoter pour obtenir le format désiré
df_pivoted = df_melted.pivot_table(index='pays', columns='année', values='valeur', aggfunc='first')

# Remplacer NaN par la dernière valeur disponible pour chaque pays (forward fill)
df_pivoted = df_pivoted.ffill(axis=1)

# Réinitialiser l'index pour avoir un DataFrame standard
df_pivoted.reset_index(inplace=True)

# Afficher le résultat
print(df_pivoted)

année                pays       2012       2013       2014        2015  \
0               Allemagne  6342394.0  6643699.0  7015236.0   7539774.0   
1                Autriche   946610.0   999095.0  1060446.0   1139736.0   
2                Belgique  1197598.0  1222741.0  1237696.0   1272751.0   
3                Bulgarie    39432.0    44506.0    53253.0     63879.0   
4                  Chypre   172427.0   170076.0   159336.0    144599.0   
5                 Croatie        NaN    24142.0    23801.0     24125.0   
6                Danemark   358714.0   374569.0   397221.0    422492.0   
7               EU27_2020        NaN        NaN        NaN  29542079.0   
8                 Espagne  5236030.0  5064584.0  4676352.0   4453985.0   
9                 Estonie   206558.0   197141.0   194917.0    197611.0   
10               Finlande   181697.0   194250.0   206651.0    218803.0   
11                 France  3944725.0  4092106.0  4227226.0   4431376.0   
12                  Grèce   921447.0  

In [14]:
# Regrouper par année pour chaque pays
for year in range(2013, 2024):  # De 2013 à 2023
    print(f"Données pour l'année {year}:")
    
    # Sélectionner uniquement les colonnes pertinentes
    df_year = df_pivoted[['pays', str(year)]]
    df_year.rename(columns={str(year): 'valeur'}, inplace=True)

     # Réinitialiser l'index pour éviter les indices numériques
    df_year.reset_index(drop=True, inplace=True)
    
    # Afficher les données
    print(df_year.head())  # Utiliser head() pour afficher les premières lignes
    print("\n")  # Ajoute un saut de ligne pour la lisibilité

Données pour l'année 2013:
année       pays     valeur
0      Allemagne  6643699.0
1       Autriche   999095.0
2       Belgique  1222741.0
3       Bulgarie    44506.0
4         Chypre   170076.0


Données pour l'année 2014:
année       pays     valeur
0      Allemagne  7015236.0
1       Autriche  1060446.0
2       Belgique  1237696.0
3       Bulgarie    53253.0
4         Chypre   159336.0


Données pour l'année 2015:
année       pays     valeur
0      Allemagne  7539774.0
1       Autriche  1139736.0
2       Belgique  1272751.0
3       Bulgarie    63879.0
4         Chypre   144599.0


Données pour l'année 2016:
année       pays     valeur
0      Allemagne  8651958.0
1       Autriche  1260188.0
2       Belgique  1327421.0
3       Bulgarie    71491.0
4         Chypre   139606.0


Données pour l'année 2017:
année       pays     valeur
0      Allemagne  9219989.0
1       Autriche  1333239.0
2       Belgique  1346358.0
3       Bulgarie    76562.0
4         Chypre   140384.0


Données pour l'

C:\Users\hp\AppData\Local\Temp\ipykernel_4452\2128398066.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year.rename(columns={str(year): 'valeur'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_4452\2128398066.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year.rename(columns={str(year): 'valeur'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_4452\2128398066.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [15]:
for year in range(2013, 2024):  # De 2013 à 2023
    # Récupérer les données pour l'année spécifique
    df_year = df_pivoted[['pays', str(year)]].copy()  # Créer une copie explicite
    df_year.rename(columns={str(year): 'valeur'}, inplace=True)
    
    # Enregistrer le DataFrame dans un fichier CSV
    file_name = f"population_par_citoyenneté_donnees_{year}.csv"  # Nom du fichier pour l'année
    df_year.to_csv(file_name, index=False, encoding='utf-8')
    print(f"Fichier CSV généré : {file_name}")

Fichier CSV généré : population_par_citoyenneté_donnees_2013.csv
Fichier CSV généré : population_par_citoyenneté_donnees_2014.csv
Fichier CSV généré : population_par_citoyenneté_donnees_2015.csv
Fichier CSV généré : population_par_citoyenneté_donnees_2016.csv
Fichier CSV généré : population_par_citoyenneté_donnees_2017.csv
Fichier CSV généré : population_par_citoyenneté_donnees_2018.csv
Fichier CSV généré : population_par_citoyenneté_donnees_2019.csv
Fichier CSV généré : population_par_citoyenneté_donnees_2020.csv
Fichier CSV généré : population_par_citoyenneté_donnees_2021.csv
Fichier CSV généré : population_par_citoyenneté_donnees_2022.csv
Fichier CSV généré : population_par_citoyenneté_donnees_2023.csv
